In [1]:
import os
import sys
import numpy as np
import pandas as pd
import csv as csv
import tqdm.auto as tqdm
from multiprocessing import Pool
from importlib import reload  
import support as sup
reload(sup)
sys.path.insert(0, '../icenumerics')

import icenumerics as ice
from icenumerics.geometry import ordering 
import magcolloids as mgc
from string import Template


import matplotlib.pyplot as plt
import time
ureg = ice.ureg

In [2]:
output = "/home/carolina/CairoLattice_ACI_PROJECT/Scripts/Output_Script-5.0"

# Cairo Lattice 

    In this script I will run the preliminarly simulations. I will run simulations in parallel and I will also extract the vertices pandas dataframes to analyze the vertex evolution in front of the interaction strenght.

In [3]:
def do_everything(exp_entry):
    
    e = exp_entry[1].e 
    l = exp_entry[1].l
    
    # <To change the seed of the thermal noise>
    
    np.random.seed()
    
    # <Introduce the parameters for the simulation>

    sp = sup.spins()
    ureg = ice.ureg
    sp.create_lattice(geometry = "cairo",size =[l,l],lattice_constant=19.5458*ureg.um, border="closed spin")
    sp.order_spins(ordering.random_ordering)

    particle = ice.particle(radius = 5*ureg.um,
                 susceptibility = 0.005,
                 diffusion = 0.125*ureg.um**2/ureg.s,
                 temperature = 300*ureg.K,
                 density = 1000*ureg.kg/ureg.m**3)

    trap_long = ice.trap(trap_sep = 10*ureg.um,
                   height = 30*ureg.pN*ureg.nm,
                   stiffness = 6e-4*ureg.pN/ureg.nm)

    trap_short = ice.trap(trap_sep = 4.5258*ureg.um,
                   height = 30*ureg.pN*ureg.nm,
                   stiffness = 6e-4*ureg.pN/ureg.nm)

    traps = []

    # <In this loop, I will define which traps have to be shortere>

    for s in sp:

        if ( s.direction[0].magnitude == 1.0 or s.direction[0].magnitude == -1.0 or 
            s.direction[1].magnitude == 1.0 or s.direction[1].magnitude == -1.0 ):

            traps.append(trap_short)

        else:

            traps.append(trap_long) 

    col = ice.colloidal_ice(sp, particle, traps, height_spread = 0, susceptibility_spread = 0.1)

    col.pad_region(30*ureg.um) # Why is this important?? Whitout this line the simulation do not run.  
    
    # <Introduce the simulation parameters>
    
    world = ice.world(
        field = 25*ureg.mT,
        temperature = 300*ureg.K,
        dipole_cutoff = 200*ureg.um)
    
    change_m_time = 1200*ureg.s
    total_time = change_m_time*2
    col.simulate(world,
                 name = "ACI_CairoLattice_25mT-2400s_1cycle_HigherHill_l%u_exp%u"%(l,e),
                 include_timestamp = False,
                 targetdir = output,
                 framerate = 1*ureg.Hz,
                 timestep = 10*ureg.ms,
                 run_time = total_time,
                 output = ["x","y","z","mux","muy","muz"])

    string_field = "(time<$Qt)*v_Bmag*time/%f+(time>=$Qt)*v_Bmag*$Qt/$Qt-(time>=$Qt)*v_Bmag*(time-$Qt)/$Qt"
    field_s = Template(string_field)
    field_d = field_s.substitute(Qt = change_m_time.to(ureg.us).magnitude)
    col.sim.field.fieldz = field_d%(change_m_time.to(ureg.us).magnitude)
    col.run_simulation()
    
#     # <Load simulation and compute vertices dataframes>
    
#     col.load_simulation(slice(0,None,1))

#     frames = col.trj.index.get_level_values("frame").unique()

#     v_df = []

#     for f in tqdm.tqdm(frames[::1]):
#         col.set_state_from_frame(f)
#         v = ice.vertices()
#         v = v.colloids_to_vertices(col)

#         v_df.append(v.vertices)

#     v_df = pd.concat(v_df, keys=frames[::1], names = ["frame"])
    
#     # <Save the dataframe and create index of the runned simulation>
    
#     v_df.to_csv(os.path.join(output,"ACI_CairoLattice_20mT-1200s_l%u_exp%u"%(l,e)+".dat"), sep='\t')
    
    name = os.path.split(col.sim.base_name)[1]
    with open(os.path.join(output,"index_Cairo_25mT-2400s_1cycle_HigherHill.dat"),'a',newline='') as file:
        writer = csv.writer(file,delimiter='\t')
        writer.writerow([name, l, e])

In [4]:
e =  np.arange(0,10)
l =  [10]
L, E = np.meshgrid(l,e)
experiments = pd.DataFrame({"e":E.flatten(),"l":L.flatten()})
experiments

,e,l
0,0,10
1,1,10
2,2,10
3,3,10
4,4,10
5,5,10
6,6,10
7,7,10
8,8,10
9,9,10


In [5]:
# %%time
if __name__ ==  '__main__': 
    num_processors = 10
    p=Pool(processes = num_processors)
    
    ## Create index text file
    if not os.path.exists(output):
        os.makedirs(output)
    with open(os.path.join(output,"index_Cairo_25mT-2400s_1cycle_CondD.dat"),'w',newline='') as file:
        writer = csv.writer(file,delimiter='\t')
        writer.writerow(["filename", "l", "exp"])
        
    list(tqdm.tqdm(p.imap(do_everything, experiments.iterrows()), total=len(experiments)))

/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:62: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new[:,0] = centers_toAddX


/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:62: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new[:,0] = centers_toAddX
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:62: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new[:,0] = centers_toAddX
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:63: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new[:,1] = centers_toAddY
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:63: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new[:,1] = centers_toAddY
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:63: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new[:,1] = centers_toAddY
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/support.py:170: UnitStrippedWarning: The unit of the quantity 

scanning file


scanning file


scanning file


...

scanning file


scanning file


scanning file


scanning file


scanning file
